# Step 2: K-Fold Cross-Validation

## What is K-Fold Cross-Validation?

It's a technique to test our model robustly. Instead of one train/test split, we do multiple splits to get a more reliable performance score.

### How it Works:
1.  **Split**: We split our training data into 'K' equal parts (or 'folds'). A common choice for K is 5 or 10.
2.  **Train & Test**: We train the model K times. Each time, we use one fold for testing and the remaining K-1 folds for training.
3.  **Average Score**: We calculate the performance score for each round and then take the average. This gives us a much more stable evaluation of our model.

### Mnemonic: **KFOLD**
*   **K** - **K** parts to split the data into.
*   **F** - **For** each part...
*   **O** - **One** part is for testing.
*   **L** - **Leftover** parts are for training.
*   **D** - **Determine** the average score.

## 1. Import Dependencies

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, accuracy_score

## 2. Load Processed Data

In [ ]:
X_train = np.load('artifacts/X_train.npz')['arr_0']
Y_train = np.load('artifacts/Y_train.npz')['arr_0']
X_test = np.load('artifacts/X_test.npz')['arr_0']
Y_test = np.load('artifacts/Y_test.npz')['arr_0']

print("Data loaded successfully from artifacts.")

## 3. K-Fold Cross-Validation Setup

We will use **StratifiedKFold**. This is a special type of K-Fold that makes sure each 'fold' has the same percentage of 'Churn' and 'Not Churn' samples as the whole dataset. It's very important for imbalanced data like ours.

In [ ]:
cv = StratifiedKFold(
    n_splits=6,
    random_state=42,
    shuffle=True
)

model_lr = LogisticRegression(
    random_state=42,
    max_iter=1000
)

## 4. Perform Cross-Validation and Evaluate

Let's run the cross-validation process for different scoring metrics.

In [ ]:
score_metrics = ['accuracy', 'precision', 'recall', 'f1']
results = {}

for s in score_metrics:
    cv_results = cross_validate(
        model_lr,
        X_train,
        Y_train,
        cv=cv,
        scoring=s
    )
    results[s] = cv_results['test_score']

print("--- Average Cross-Validation Metrics ---")
results_df = pd.DataFrame(results).mean().to_frame('Average Score')
display(results_df.style.format('{:.4f}').background_gradient(cmap='viridis'))

## 5. Train Final Model and Evaluate on Test Set

Cross-validation gives us confidence in our model. Now, we train the model on the **entire training set** and make our final evaluation on the **unseen test set**.

In [ ]:
# Train the final model on the full training data
final_model = LogisticRegression(
    random_state=42,
    max_iter=1000
)
final_model.fit(X_train, Y_train)

# Make predictions on the test set
Y_hat_test = final_model.predict(X_test)

In [ ]:
accuracy = accuracy_score(Y_test, Y_hat_test)
precision = precision_score(Y_test, Y_hat_test)
recall = recall_score(Y_test, Y_hat_test)
f1 = f1_score(Y_test, Y_hat_test)

print("--- Final Model Performance on Test Set ---")
print(f"| Metric    | Score   |")
print(f"|-----------|---------|")
print(f"| Accuracy  | {accuracy:.4f}  |")
print(f"| Precision | {precision:.4f}  |")
print(f"| Recall    | {recall:.4f}  |")
print(f"| F1-Score  | {f1:.4f}  |")
print("---------------------------------")

# Display the confusion matrix
cm = confusion_matrix(Y_test, Y_hat_test)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Not Churned', 'Churned'], 
            yticklabels=['Not Churned', 'Churned'])
plt.xlabel('Predicted Label')
plt.ylabel('Actual Label')
plt.title('Confusion Matrix for Test Set')
plt.show()